# gpu 환경 설정 

In [ ]:
# 할당된 gpu보기
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [ ]:
# 사용가능한 gpu 목록
# 현재 할당된 것이 cpu인지 gpu인지 확인 
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
# 할당된 gpu를 실행 
import os 

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus=tf.config.experimental.list_physical_devices("GPU")

if gpus:
  try:
    # GPU 메모리 사용을 설정 
    tf.config.experimental.set_memory_growth(gpus[0],True)

  except RuntimeError as re:
    print(re)

In [ ]:
# gpu 메모리를 전부 할당하지 않고 필요에 따라 자동으로 할당하도록 설정 
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth=True
session=InteractiveSession(config=config)

# 공유 드라이브에 있는 파일 가져오기 (드라이브 마운트)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 드라이브에서 이미지 파일 가져와서 정리하는 코드 

In [ ]:
#1. 이미지 데이터가 압축 파일 형태로 저장되어있어서 읽어올때 풀어야 한다. (damage)

from zipfile import ZipFile as zf
import numpy as np
from PIL import Image
from io import BytesIO
import os


zf("/content/drive/MyDrive/Data/160. 차량파손 이미지 데이터/01.데이터/1.Training/1.원천데이터/TS_damage.zip",).extractall("/content/damage_image")

In [ ]:
#1. 이미지 데이터가 압축 파일 형태로 저장되어있어서 읽어올때 풀어야 한다. (damage_part)

from zipfile import ZipFile as zf
import numpy as np
from PIL import Image
from io import BytesIO
import os


zf("/content/drive/MyDrive/Data/160. 차량파손 이미지 데이터/01.데이터/1.Training/1.원천데이터/TS_damage_part.zip",).extractall("/content/damage_part_image")

## validation 이미지 데이터도 마찬가지로 가져와서 정리


In [ ]:
from zipfile import ZipFile as zf
import numpy as np
from PIL import Image
from io import BytesIO
import os


zf("/content/drive/MyDrive/Data/160. 차량파손 이미지 데이터/01.데이터/2.Validation/1.원천데이터/VS_damage.zip",).extractall("/content/val/damage_image")

In [ ]:
from zipfile import ZipFile as zf
import numpy as np
from PIL import Image
from io import BytesIO
import os


zf("/content/drive/MyDrive/Data/160. 차량파손 이미지 데이터/01.데이터/2.Validation/1.원천데이터/VS_damage_part.zip",).extractall("/content/val/damage_part_image")

# 라벨링 데이터를 가져와서 정리하는 코드

In [ ]:
from zipfile import ZipFile as zf
import numpy as np
from PIL import Image
from io import BytesIO
import os


zf("/content/drive/MyDrive/Data/160. 차량파손 이미지 데이터/01.데이터/1.Training/2.라벨링데이터/TL_damage.zip",).extractall("/content/label/damage_image")

In [ ]:

from zipfile import ZipFile as zf
import numpy as np
from PIL import Image
from io import BytesIO
import os


zf("/content/drive/MyDrive/Data/160. 차량파손 이미지 데이터/01.데이터/1.Training/2.라벨링데이터/TL_damage_part.zip",).extractall("/content/label/damage_part_image")

## validation 라벨링 데이터 정리하기

In [ ]:
#damage 라벨링
from zipfile import ZipFile as zf
import numpy as np
from PIL import Image
from io import BytesIO
import os


zf("/content/drive/MyDrive/Data/160. 차량파손 이미지 데이터/01.데이터/2.Validation/2.라벨링데이터/VL_damage.zip",).extractall("/content/label/val-damage_image")

In [ ]:
#damage_part
from zipfile import ZipFile as zf
import numpy as np
from PIL import Image
from io import BytesIO
import os


zf("/content/drive/MyDrive/Data/160. 차량파손 이미지 데이터/01.데이터/2.Validation/2.라벨링데이터/VL_damage_part.zip",).extractall("/content/label/val-damage_part_image")

# dataset 디렉토리를 만드는 코드 

In [ ]:
%pwd

'/content'

In [ ]:
%mkdir dataset

In [ ]:
%cd dataset

/content/dataset


In [ ]:
%mkdir train

In [ ]:
%mkdir val

In [ ]:
%cd train

/content/dataset/train


In [ ]:
%mkdir images

In [ ]:
%mkdir labels

In [ ]:
%cd ../

/content/dataset


In [ ]:
%pwd

'/content/dataset/validation'

In [ ]:
%mkdir images

In [ ]:
%mkdir labels

# labeling json 파일 변환하는 코드 

In [ ]:
import os
from glob import glob

# damage_labeling
label_list_json_dir1="/content/label/damage_image/damage/"
label_list_json_file1=os.listdir(label_list_json_dir)

In [ ]:
len(label_list_json_file1)

403559

In [ ]:
import os
from glob import glob

# damage_part_labeling
label_list_json_dir2="/content/label/damage_part_image/damage_part"
label_list_json_file2=os.listdir(label_list_json_dir2)

In [ ]:
len(label_list_json_file2)

109062

In [ ]:
import json

for file in label_list_json_file1 :
    with open(label_list_json_dir1 +"/"+file,"r") as f :
        json_data = json.load(f)
        
        f_name2 = json_data["images"]["file_name"][:-4]
        width = json_data["images"]["width"]  # 이미지 너비
        height = json_data["images"]["height"]  # 이미지 높이
        
        with open(f"/content/label/damage_image/{f_name2}.txt", 'w') as f1 :
           for d in json_data['annotations']:
                x1, y1, x2, y2 = d["bbox"]  # bbox의 왼쪽 상단(x1, y1), 오른쪽하단(x2, y2 좌표)
                x, y = (x1 + (x1 + x2)) / 2 / width,(y1 + (y1-y2)) / 2 / height  # YOLO 모델에 넣을 이미지에 대한 bbox 중앙점의 상대적 좌표(x, y) 
                w, h = x2 / width, y2 / height  # YOLO 모델에 넣을 bbox의 이미지에 대한 상대적 너비, 높이

                if d["damage"] == None:
                    num = 0
                elif d["damage"] == 'Crushed':
                    num = 1
                elif d["damage"] == 'Scratched':
                    num = 2
                elif d["damage"] == 'Breakage':
                    num = 3
                elif d["damage"] == 'Separated':
                    num = 4 

                bbox = ' '.join(map(str, [num, x, y, w, h]))

                if d["damage"] == None :
                    f1.write(f'0 {bbox}\n')  # txt파일에 클래스 인덱스 0 bbox값(x y w h) 쓰기                            

                # damage가 null이 아니면,
                else :
                    f1.write(f'{bbox}\n') # txt파일에 클래스 인덱스와 bbox값(x y w h) 쓰기   

In [ ]:
import json

for file in label_list_json_file2 :
    with open(label_list_json_dir2 +"/"+file,"r") as f :
        json_data = json.load(f)
        
        f_name2 = json_data["images"]["file_name"][:-4]
        width = json_data["images"]["width"]  # 이미지 너비
        height = json_data["images"]["height"]  # 이미지 높이
        
        with open(f"/content/label/damage_part_image/{f_name2}.txt", 'w') as f1 :
           for d in json_data['annotations']:
                x1, y1, x2, y2 = d["bbox"]  # bbox의 왼쪽 상단(x1, y1), 오른쪽하단(x2, y2 좌표)
                x, y = (x1 + (x1 + x2)) / 2 / width,(y1 + (y1-y2)) / 2 / height  # YOLO 모델에 넣을 이미지에 대한 bbox 중앙점의 상대적 좌표(x, y) 
                w, h = x2 / width, y2 / height  # YOLO 모델에 넣을 bbox의 이미지에 대한 상대적 너비, 높이

                if d["damage"] == None:
                    num = 0
                elif d["damage"] == 'Crushed':
                    num = 1
                elif d["damage"] == 'Scratched':
                    num = 2
                elif d["damage"] == 'Breakage':
                    num = 3
                elif d["damage"] == 'Separated':
                    num = 4 

                bbox = ' '.join(map(str, [num, x, y, w, h]))

                if d["damage"] == None :
                    f1.write(f'0 {bbox}\n')  # txt파일에 클래스 인덱스 0 bbox값(x y w h) 쓰기                            

                # damage가 null이 아니면,
                else :
                    f1.write(f'{bbox}\n') # txt파일에 클래스 인덱스와 bbox값(x y w h) 쓰기   

# val 데이터도 전부 라벨링 변환시켜준다. 

In [ ]:
import os
from glob import glob

# damage_labeling
label_list_json_dir3="/content/label/val-damage_image/damage"
label_list_json_file3=os.listdir(label_list_json_dir3)

In [ ]:
import os
from glob import glob

# damage_labeling
label_list_json_dir4="/content/label/val-damage_part_image/damage_part"
label_list_json_file4=os.listdir(label_list_json_dir4)

In [ ]:
import json

for file in label_list_json_file3 :
    with open(label_list_json_dir3 +"/"+file,"r") as f :
        json_data = json.load(f)
        
        f_name2 = json_data["images"]["file_name"][:-4]
        width = json_data["images"]["width"]  # 이미지 너비
        height = json_data["images"]["height"]  # 이미지 높이
        
        with open(f"/content/label/val-damage_image/{f_name2}.txt", 'w') as f1 :
           for d in json_data['annotations']:
                x1, y1, x2, y2 = d["bbox"]  # bbox의 왼쪽 상단(x1, y1), 오른쪽하단(x2, y2 좌표)
                x, y = (x1 + (x1 + x2)) / 2 / width,(y1 + (y1-y2)) / 2 / height  # YOLO 모델에 넣을 이미지에 대한 bbox 중앙점의 상대적 좌표(x, y) 
                w, h = x2 / width, y2 / height  # YOLO 모델에 넣을 bbox의 이미지에 대한 상대적 너비, 높이

                if d["damage"] == None:
                    num = 0
                elif d["damage"] == 'Crushed':
                    num = 1
                elif d["damage"] == 'Scratched':
                    num = 2
                elif d["damage"] == 'Breakage':
                    num = 3
                elif d["damage"] == 'Separated':
                    num = 4 

                bbox = ' '.join(map(str, [num, x, y, w, h]))

                if d["damage"] == None :
                    f1.write(f'0 {bbox}\n')  # txt파일에 클래스 인덱스 0 bbox값(x y w h) 쓰기                            

                # damage가 null이 아니면,
                else :
                    f1.write(f'{bbox}\n') # txt파일에 클래스 인덱스와 bbox값(x y w h) 쓰기   

In [ ]:
import json

for file in label_list_json_file4 :
    with open(label_list_json_dir4 +"/"+file,"r") as f :
        json_data = json.load(f)
        
        f_name2 = json_data["images"]["file_name"][:-4]
        width = json_data["images"]["width"]  # 이미지 너비
        height = json_data["images"]["height"]  # 이미지 높이
        
        with open(f"/content/label/val-damage_part_image/{f_name2}.txt", 'w') as f1 :
           for d in json_data['annotations']:
                x1, y1, x2, y2 = d["bbox"]  # bbox의 왼쪽 상단(x1, y1), 오른쪽하단(x2, y2 좌표)
                x, y = (x1 + (x1 + x2)) / 2 / width,(y1 + (y1-y2)) / 2 / height  # YOLO 모델에 넣을 이미지에 대한 bbox 중앙점의 상대적 좌표(x, y) 
                w, h = x2 / width, y2 / height  # YOLO 모델에 넣을 bbox의 이미지에 대한 상대적 너비, 높이

                if d["damage"] == None:
                    num = 0
                elif d["damage"] == 'Crushed':
                    num = 1
                elif d["damage"] == 'Scratched':
                    num = 2
                elif d["damage"] == 'Breakage':
                    num = 3
                elif d["damage"] == 'Separated':
                    num = 4 

                bbox = ' '.join(map(str, [num, x, y, w, h]))

                if d["damage"] == None :
                    f1.write(f'0 {bbox}\n')  # txt파일에 클래스 인덱스 0 bbox값(x y w h) 쓰기                            

                # damage가 null이 아니면,
                else :
                    f1.write(f'{bbox}\n') # txt파일에 클래스 인덱스와 bbox값(x y w h) 쓰기   

# 이미지 데이터 전부 복사해서 dataset train image에 옯기기 

In [ ]:
from glob import glob
import shutil

img_list1=glob("/content/damage_image/damage/*.jpg")

for i in img_list1:
  shutil.move(i, '/content/dataset/train/images' )

In [ ]:
from glob import glob
import shutil

img_list2=glob("/content/damage_part_image/damage_part/*.jpg")

for i in img_list2:
  shutil.move(i, '/content/dataset/train/images' )

In [ ]:
## 제대로 이동이 되었는지 확인해볼까?

dataset_check=glob('/content/dataset/train/images/*.jpg')

len(dataset_check)

402143

## validataion 이미지를 이동시키기

In [ ]:
img_list3=glob("/content/val/damage_image/damage/*.jpg")

for i in img_list3:
  shutil.move(i, '/content/dataset/validation/images' )

In [ ]:
img_list4=glob("/content/val/damage_part_image/damage_part/*.jpg")

for i in img_list4:
  shutil.move(i, '/content/dataset/validation/images' )

In [ ]:
## 제대로 이동이 되었는지 체크해보기 

dataset_check2=glob('/content/dataset/validation/images/*.jpg')

len(dataset_check2)

67693

## labels 데이터 긁어오기 

In [ ]:

label_list1=glob("/content/label/damage_image/*.txt")

for i in label_list1:
  shutil.move(i, "/content/dataset/train/labels")

In [ ]:
label_list2=glob("/content/label/damage_part_image/*.txt")

for i in label_list2:
  shutil.move(i, "/content/dataset/train/labels")

In [ ]:
label_list3=glob("/content/label/val-damage_image/*.txt")

for i in label_list3:
  shutil.move(i, "/content/dataset/validation/labels")

In [ ]:
label_list4=glob("/content/label/val-damage_part_image/*.txt")

for i in label_list4:
  shutil.move(i, "/content/dataset/validation/labels")

# dataset 다시 확인 

In [ ]:
dataset1=glob("/content/dataset/train/images/*.jpg")

len(dataset1)

511205

In [ ]:
dataset2=glob("/content/dataset/train/labels/*.txt")

len(dataset2)

512621

In [ ]:
dataset3=glob("/content/dataset/validation/images/*.jpg")

len(dataset3)

67693

In [ ]:
dataset4=glob("/content/dataset/validation/labels/*.txt")

len(dataset4)

67693

# data.yaml 파일 만들기 



In [ ]:
data={}# 빈 딕트 

data["train"]="/content/dataset/train/images"
data["val"]="/content/dataset/val/images"

data['nc']= 5
data['names']=['None',
'Crushed',
'Scratched',
'Breakage',
'Separated']

In [ ]:
import  yaml

with open('/content/dataset/data.yaml','w') as f:
    yaml.dump(data,f)

# yolov5와 yolov7 모델 클론하기

In [ ]:
# yolo v5
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 14974, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14974 (delta 6), reused 9 (delta 3), pack-reused 14960
Receiving objects: 100% (14974/14974), 13.65 MiB | 26.13 MiB/s, done.
Resolving deltas: 100% (10344/10344), done.


In [ ]:
#yolo v7 
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 998, done.
remote: Total 998 (delta 0), reused 0 (delta 0), pack-reused 998
Receiving objects: 100% (998/998), 69.77 MiB | 29.44 MiB/s, done.
Resolving deltas: 100% (467/467), done.


# yolov7 모델 돌려보기 

In [ ]:
%pwd

'/content/dataset/val'

In [ ]:
%cd /content/yolov7

/content/yolov7


In [ ]:
# yolo v7 모델 돌리기 
# 성민씨 코드 
# 학습 모듈 실행

#경고 무시
#import warnings
#warnings.filterwarnings("ignore")
# 사용할 파이썬 파일
python_t = 'train.py'
# 사용할 데이터를 적어둔 yaml file
data = '/content/dataset/data.yaml'
# 사용할 모델 구조 (x>l>n>m>s)
cfg = '/content/yolov7/cfg/training/yolov7.yaml'

# 사용할 가중치
weights = './yolov7.pt'
# 학습 결과를 저장할 폴더 이름
name = '/content/yolov7/car_destroy/yolov7_val.'

!python {python_t} --batch 16 --epochs 20 --data {data} --cfg {cfg} --weights {weights} --name {name}

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
val: WARNING: Ignoring corrupted image and/or label /content/dataset/val/images/0559690_as-0009398.jpg: negative labels
val: WARNING: Ignoring corrupted image and/or label /content/dataset/val/images/0559694_as-0073537.jpg: negative labels
val: WARNING: Ignoring corrupted image and/or label /content/dataset/val/images/0559695_as-0073537.jpg: negative labels
val: WARNING: Ignoring corrupted image and/or label /content/dataset/val/images/0559696_as-0073537.jpg: negative labels
val: WARNING: Ignoring corrupted image and/or label /content/dataset/val/images/0559698_as-3149926.jpg: negative labels
val: WARNING: Ignoring corrupted image and/or label /content/dataset/val/images/0559701_as-2803939.jpg: negative labels
val: Scanning '/content/dataset/val/labels' images and labels... 57814 found, 0 missing, 0 empty, 30599 corrupted:  85% 57814/67693 [01:14<00:11, 833.30it/s]val: WARNING: Ignoring corrupted image and/or label /content/dataset/val/images/0559702